<a href="https://colab.research.google.com/github/riyalguha/collaborative_filtering_recc_system/blob/main/SVD_matrix_collaborative_filtering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from sklearn.decomposition import TruncatedSVD

In [ ]:
columns = ['user_id','item_id','rating','timestamp']
frame = pd.read_csv('u.data',sep='\t',names = columns)

In [ ]:
frame.head()

,user_id,item_id,rating,timestamp
0,196,242,3.0,881250949.0
1,186,302,3.0,891717742.0
2,22,377,1.0,878887116.0
3,244,51,2.0,880606923.0
4,166,346,1.0,886397596.0


In [ ]:
frame['rating'].describe()

count    53153.000000
mean         3.536451
std          1.135215
min          1.000000
25%          3.000000
50%          4.000000
75%          4.000000
max          5.000000
Name: rating, dtype: float64

In [ ]:
columns = ['item_id', 'movie title', 'release date', 'video release date', 'IMDb URL', 'unknown', 'Action', 'Adventure',
          'Animation', 'Childrens', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror',
          'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']

In [ ]:
movies = pd.read_csv('u.item',sep='|',names = columns, encoding = 'latin-1')

In [ ]:
movie_names = movies[['item_id','movie title']]

In [ ]:
movies.head()

,item_id,movie title,release date,video release date,IMDb URL,unknown,Action,Adventure,Animation,Childrens,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,...,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [ ]:
movie_names.head()

,item_id,movie title
0,1,Toy Story (1995)
1,2,GoldenEye (1995)
2,3,Four Rooms (1995)
3,4,Get Shorty (1995)
4,5,Copycat (1995)


In [21]:
combined_frame = pd.merge(frame,movie_names,on='item_id')

In [22]:
combined_frame.head()

,user_id,item_id,rating,timestamp,movie title
0,196,242,3.0,881250949.0,Kolya (1996)
1,63,242,3.0,875747190.0,Kolya (1996)
2,226,242,5.0,883888671.0,Kolya (1996)
3,154,242,3.0,879138235.0,Kolya (1996)
4,306,242,5.0,876503793.0,Kolya (1996)


In [23]:
combined_frame.groupby('item_id')['rating'].count().sort_values(ascending = False).head()

item_id
50     301
258    279
100    268
288    266
181    259
Name: rating, dtype: int64

In [24]:
combined_frame.head()

,user_id,item_id,rating,timestamp,movie title
0,196,242,3.0,881250949.0,Kolya (1996)
1,63,242,3.0,875747190.0,Kolya (1996)
2,226,242,5.0,883888671.0,Kolya (1996)
3,154,242,3.0,879138235.0,Kolya (1996)
4,306,242,5.0,876503793.0,Kolya (1996)


In [27]:
Filter = combined_frame['item_id']==50

In [30]:
combined_frame[Filter]['movie title'].unique()

array(['Star Wars (1977)'], dtype=object)

## Building Utility Matrix

In [39]:
rating_crosstab = combined_frame.pivot_table(values='rating',index='user_id',columns='movie title',fill_value = 0)

In [40]:
rating_crosstab.head()

movie title,'Til There Was You (1997),1-900 (1994),101 Dalmatians (1996),12 Angry Men (1957),187 (1997),2 Days in the Valley (1996),"20,000 Leagues Under the Sea (1954)",2001: A Space Odyssey (1968),3 Ninjas: High Noon At Mega Mountain (1998),"39 Steps, The (1935)",...,"Wrong Trousers, The (1993)",Wyatt Earp (1994),Yankee Zulu (1994),Year of the Horse (1997),Young Frankenstein (1974),Young Guns (1988),Young Guns II (1990),"Young Poisoner's Handbook, The (1995)",Zeus and Roxanne (1997),unknown
user_id,,,,,,,,,,,,,,,,,,,,,
1,0,0,2,5,0,0,3,4,0,0,...,5,0,0,0,5,3,0,0,0,4
2,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,2,0,0,0,0,4,0,0,...,0,0,0,0,4,0,0,0,0,4


## Transposing the Matrix

In [41]:
rating_crosstab.shape

(801, 1588)

In [42]:
X = rating_crosstab.values.T

In [43]:
X.shape

(1588, 801)

## Decomposing the Matrix

In [45]:
SVD = TruncatedSVD(n_components = 12, random_state = 17)

In [46]:
resultant_matrix = SVD.fit_transform(X)

In [47]:
resultant_matrix.shape

(1588, 12)

## Generating correlation Matrix

In [48]:
corr_mat = np.corrcoef(resultant_matrix)

In [49]:
corr_mat.shape

(1588, 1588)

In [60]:
movies_names = rating_crosstab.columns

In [61]:
movies_names.shape

(1588,)

## Isolating Star Wars from the matrix

In [63]:
movies_list = list(movies_names)

In [64]:
star_wars = movies_list.index('Star Wars (1977)')

In [65]:
print(star_wars)

1339


In [66]:
corr_star_wars = corr_mat[star_wars]

In [70]:
list(movies_names[(corr_star_wars <1.0) & (corr_star_wars >0.8)])

['Babe (1995)',
 'Back to the Future (1985)',
 'Blues Brothers, The (1980)',
 'Empire Strikes Back, The (1980)',
 'Frankie Starlight (1995)',
 'Godfather, The (1972)',
 'Great Escape, The (1963)',
 'Hoop Dreams (1994)',
 'Live Nude Girls (1995)',
 'Lone Star (1996)',
 'Men in Black (1997)',
 'Moll Flanders (1996)',
 'Muppet Treasure Island (1996)',
 'Return of the Jedi (1983)',
 'Searching for Bobby Fischer (1993)',
 'Star Wars (1977)']